## AISE4010- Assignment2 - Time Series Forecasting using CNN, RNN, LSTM and GRU

## Grade: 100 points

### Instructions

#### Follow These Steps before submitting your assignment

1. Complete the notebook.

2. Make sure all plots have axis labels.

3. Once the notebook is complete, `Restart` your kernel by clicking 'Kernel' > 'Restart & Run All'.

4. Fix any errors until your notebook runs without any problems.

5. Submit one completed notebook for the group to OWL by the deadline.

6. Make sure to reference all external code and documentation used.

### Dataset

The `StockData.csv` represents a time series dataset designed to simulate the behavior of stock prices over a period of 1000 days, starting from January 1, 2023. This dataset includes key features commonly found in stock market data, such as opening prices, closing prices, daily highs, daily lows, and trading volume.

Features:
- Date: The timestamp representing the specific day for each record.
- Open: The stock's price at the start of the trading day.
- High: The highest price of the stock during the trading day. It is generally higher than the opening price, reflecting intraday market movements.
- Low: The lowest price of the stock during the trading day. It is usually lower than the opening price, capturing downward movements in the market.
- Close: The stock’s price at the end of the trading day. This price reflects the market's closing valuation and fluctuates within a range close to the opening price.
- Volume: The number of shares traded during the day.

### Question 1 - Data Preprocessing (15%)
- Q1.1 Load the provided dataset and plot the `Closing prices` time series. (5%)
- Q1.2 (5%):

  - a) Decompose the time series into its components and explain them.  
  - b) What do the trend and seasonality components tell you about the overall behavior of the time series? How does understanding     these components help in selecting a model for time series forecasting?
- Q1.3 Restructure the time series using a sliding window approach (with size 10) to prepare it for supervised learning. Split the `Closing prices` time series into 80% for training and 20% for testing. (5%)


In [1]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('StockData.csv')

print("Q1.1 — Missing Values Summary\n")

dfNull = df.isnull().sum()

print(dfNull)

if dfNull.sum() == 0:
  print("\nNo missing values detected — proceeding without any cleaning.\n")
else :
  print("\nMissing values detected")

df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)
df = df.sort_values('Date').reset_index(drop=True)

ts = df.set_index('Date')[['Close']]

print("Q1.1 — Dataset Summary\n")
print(f"Rows available: {len(ts)}")
print(f"Date range: {ts.index[0].date()}  →  {ts.index[-1].date()}\n")
print(ts.head(3))

plt.figure(figsize=(12, 4))
plt.plot(ts.index, ts['Close'])
plt.title("Closing Prices Over Time")
plt.xlabel("Date")
plt.ylabel("Close")
plt.grid(True)
plt.tight_layout()
plt.show()

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
import numpy as np

period = 7  # ~ one trading week

# 1) Trend: centered 7-day moving average

#   Trend = (Close_{t-3} + ... Close_{t+3}) / 7
#         - The first/last 3 rows have no centered window so Nan (expected)

trend = ts['Close'].rolling(window=period, center=True).mean()

# 2) Seasonality: average detrended value at each cycle position (0..6)
#    2.1 - Detrend: how much each point deviates from its local level (value provided around that day from the trend)
#        - Detrend = Close - Trend

detrended = ts['Close'] - trend

#    2.2 = Assign each row a cycle position p(t) = t mod 7

pos = np.arange(len(ts)) % period

#    2.3 - For each position k in {0...6}, average the dtrended values across the whole series
#        - S_index[k] = mean{Detrended | pos(t) = k}

seasonal_index = pd.Series(detrended.values, index=pos).groupby(level=0).mean()

#    2.4 - Enforce additive zero-mean seasonality over one full cycle:
#        - S_index[k] = S_index[k] - [sum_j S_index[j]]/7

seasonal_index -= seasonal_index.mean()

#    2.5 - Map the 5 indices back to every row via its position:
#        - Seasonal = S_index[pos(t)]

seasonal = seasonal_index.reindex(pos).to_numpy()

# 3) Residual

#   Residual = Close - Trend - Seasonal
#            = Residual will be NaN where Trend is NaN

residual = ts['Close'].to_numpy() - trend.to_numpy() - seasonal

# 4) Pack results for inspection / plotting
decomp = pd.DataFrame(
    {'Observed': ts['Close'].to_numpy(),   #Original series Close
     'Trend': trend.to_numpy(),            #Centered 7-day moving average
     'Seasonal': seasonal,                 #Repeating 7-step pattern
     'Residual': residual},
    index=ts.index
)

# Quick preview
print("Q1.2 — Decomposition (head):")
print(decomp.head(14))

# Analyze components for discussion

#1. Fit a line on the valid Trend to see if its rising or falling
trend_valid = decomp['Trend'].dropna()

if len(trend_valid) > 8:

    x = np.arange(len(trend_valid))
    slope = np.polyfit(x, trend_valid.values, 1)[0]
    trend_dir = "upward" if slope > 0 else ("downward" if slope < 0 else "flat")

else:

    trend_dir = "n/a"

#2. Seasonal Amplitude: Peak-to-peak size of the repeating 7-step pattern
seasonal_amp = float(np.nanmax(decomp['Seasonal']) - np.nanmin(decomp['Seasonal']))

#3. Residual std: how noisy the unexplained remainder is
resid_std = float(np.nanstd(decomp['Residual']))

print(f"\nTrend direction (slope sign): {trend_dir}")
print(f"Seasonal amplitude (peak-to-peak, 7-step cycle): ~{seasonal_amp:.3f}")
print(f"Residual std: ~{resid_std:.3f}")

# 5) Plots (simple, consistent)
plt.figure(figsize=(12, 3))
plt.plot(decomp.index, decomp['Observed'])
plt.title("Observed (Close)")
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 3))
plt.plot(decomp.index, decomp['Trend'])
plt.title("Trend (Centered 7-day Moving Average)")
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 3))
plt.plot(decomp.index, decomp['Seasonal'])
plt.title("Seasonal (7-position cycle)")
plt.grid(True)
plt.tight_layout()
plt.show()

# NEW: Zoomed seasonal plot (first 28 days ≈ 4 full cycles) for visual clarity
sample_length = 28
plt.figure(figsize=(12, 3))
plt.plot(decomp.index[:sample_length], decomp['Seasonal'][:sample_length])
plt.title(f"Seasonal (First {sample_length} Days — Zoomed, 7-position cycle)")
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 3))
plt.plot(decomp.index, decomp['Residual'])
plt.title("Residual = Observed − Trend − Seasonal")
plt.grid(True)
plt.tight_layout()
plt.show()


#**Q1.2(a)** - Decomposition Components

**Observed:** This is the raw closing price directly from the dataset - it includes the combined effect of long-term growth, short-term repetitive cycles, and noise.

**Trend:** A smoothed version of the price that captures the long-term direction of movement, ignoring short-term fluctuations. In this dataset the trend shows a gradual upward increase over time, with a significant jump around early 2025, indicating strong long term market growth rather than random fluctuation.

**Seasonal:** A repeating short-term 7-step pattern that cycles consistently over time. This reflects small but predicatble oscillations around the trend, likely due to systematic short-term behavior. The amplitude shows that seasonality influences short-term variation but does not dominate overall direction.

**Residual:** Whatever remains after the trend and seasonality are removed - it represents noise caused by market shocks, news events, or random trading activity.

#**Q1.2(b)** - Interpretation of Trend & Seasonality

**Trend:** The trend shows a strong upward long-term movement, indicating the asset is generally growin in value over timie, with a major increase starting around 2025.

**Seasonality:** The seasonal component exhibits a stable, repeating 7-day cycle, suggesting consistent short-term fluctuations that repeat regularly.

****

**Why This Matters For Forecasting**

Since there is a clear upward trend, we should use a forecasting model that supports non-stationarity. Because seasonality is present and stable, a seasonality-aware model will perfrom better than a model that assumes the data has no repeating pattern. SARIMA or LSTM could be good models to use on this dataset.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 1) Extract Close series
prices = ts['Close'].values.astype(float)

# 2) Build sliding windows (window = 10)
WINDOW = 10
X, y = [], []
for i in range(len(prices) - WINDOW):
    X.append(prices[i : i + WINDOW])   # Input: Contains 10 consecutive past closing prices (day t to t+9)
    y.append(prices[i + WINDOW])       # Output: The close immediately after those 10 days (day t+10)
X = np.array(X)
y = np.array(y)

print("Q1.3 — Raw supervised arrays")
print(f"X shape (samples, window): {X.shape}")
print(f"y shape (samples,):       {y.shape}")

# 3) 80/20 split INDEX
#    - We are not shuffling the data because the time must remain sequential

split_idx = int(len(X) * 0.8)

# 4) Normalize (GLOBAL scaling: fit on TRAIN ONLY, apply to all)
#    - MinMax to [0,1] (good for NN training stability)
#    - Fit x_scaler on all values from TRAIN windows; fit y_scaler on TRAIN targets

x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

x_scaler.fit(X[:split_idx].reshape(-1, 1))
y_scaler.fit(y[:split_idx].reshape(-1, 1))

# Transform 2D window arrays with a single scaler
def transform_windows(arr2d, scaler):
    flat = arr2d.reshape(-1, 1)
    flat_scaled = scaler.transform(flat)
    return flat_scaled.reshape(arr2d.shape)

X_scaled = transform_windows(X, x_scaler)
y_scaled = y_scaler.transform(y.reshape(-1, 1))

# 5) Final chronological split (no shuffling)
X_train = X_scaled[:split_idx].copy()
X_test  = X_scaled[split_idx:].copy()
y_train = y_scaled[:split_idx].copy()
y_test  = y_scaled[split_idx:].copy()

print("\nQ1.3 — Train/Test split (80/20 chronological)")
print(f"Training set shape: X_train {X_train.shape}, y_train {y_train.shape}")
print(f"Test set shape:     X_test  {X_test.shape},  y_test  {y_test.shape}")

### Question2 - 1D Convolutional Neural Network (CNN) (20%)
- Q2.1 Create a 1D CNN model for time series forecasting. The model should have two convolutional layers of size 64 and 32 and kernel size of 3, fully connected layer of size 50, droup out of 0.3, and ReLU activation. Train the model using the provided dataset for 50 epochs. Use the batch_size of 32, and ADAM optimizer. Plot the training loss and validation loss. (12%)
- Q2.2 Evaluate the model's performance using RMSE and MAE on the test set. Plot the predicted values alongside the actual values to visualize how well the model is forecasting the time series. (6%)
- Q2.3 How does the 1D-CNN perform in terms of capturing short-term dependencies? Based on the RMSE and MAE values, do you think this model can effectively capture long-term trends? Why or why not? (2%)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

#1) Prepare shapes for Conv1D
#   - Conv1D expects input: (samples, timesteps, features) so we must change our input to this format
#         - (1, 10, 1): Each input sample is a 10-day sequence, with 1 value per day (the closing price)
#   - We reshape our output to 1D - Keras prefers 1D targets for regression.

WINDOW = X_train.shape[1]
X_train_cnn = X_train.reshape(-1, WINDOW, 1)
X_test_cnn  = X_test.reshape(-1, WINDOW, 1)

y_train_vec = y_train.ravel()
y_test_vec  = y_test.ravel()

print("Q2.1 — Shapes ready for CNN")
print(f"X_train_cnn: {X_train_cnn.shape}  (samples, timesteps={WINDOW}, features=1)")
print(f"X_test_cnn: {X_test_cnn.shape}")
print(f"y_train: {y_train_vec.shape}")
print(f"y_test: {y_test_vec.shape}")

# 2) Chronological validation split from the training set
#   - Used to monitor learning progress during training. We are using the last 10% of training for validation
#   - Check's if the data is overfitting or underperforming. We keep the data in chronological order.

val_start = int(len(X_train_cnn) * 0.9)
X_tr, X_val = X_train_cnn[:val_start], X_train_cnn[val_start:]
y_tr, y_val = y_train_vec[:val_start],  y_train_vec[val_start:]

print("\nQ2.1 — Train/Val split (from training set)")
print(f"X_tr:  {X_tr.shape}, y_tr:  {y_tr.shape}")
print(f"X_val: {X_val.shape}, y_val: {y_val.shape}\n")

# 3) Define the 1D CNN per spec:
#    - Conv1D: 64 filters, kernel_size=3, ReLU
#    - Conv1D: 32 filters, kernel_size=3, ReLU
#    - Dense:  50 units, ReLU    -  Learns nonlinear relationships between extracted features
#    - Dropout: 0.3              -  Prevents overfitting by randomly dropping 30% of neurons
#    - Output: 1 (regression)
#    - Optimizer: Adam           -  Automatically adapts the learning rate during training.
#    - Loss: MSE

tf.random.set_seed(42)
np.random.seed(42)

model = models.Sequential([
    layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(WINDOW, 1)),
    layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dense(50, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1)
])

model.compile(optimizer=optimizers.Adam(), loss='mse', metrics=['mae'])
model.summary()

# 4) Train for 50 epochs, batch_size=32
EPOCHS = 50
BATCH_SIZE = 32

history = model.fit(
    X_tr, y_tr,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1
)

# 5) Plot training vs validation loss (MSE)
train_loss = history.history['loss']
val_loss   = history.history['val_loss']

plt.figure(figsize=(10, 4))
plt.plot(train_loss, label='Train Loss (MSE)')
plt.plot(val_loss,   label='Validation Loss (MSE)')
plt.title("Q2.1 — Training vs. Validation Loss (1D-CNN)")
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 1) Predict on the test set (still in [0,1] scale)

y_pred_scaled = model.predict(X_test_cnn, verbose=0).ravel()

# 2) Inverse-transform to original price scale
#    - To interpret RMSE/MAE properly, we reverse the normalized scaling we applied in 1.3

y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()
y_true = y_scaler.inverse_transform(y_test_vec.reshape(-1, 1)).ravel()

# 3) Metrics in original units

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae  = mean_absolute_error(y_true, y_pred)

print("Q2.2 — Test Metrics (original price scale)")
print(f"RMSE: {rmse:.4f}")
print(f"MAE : {mae:.4f}")

# 4) Plot Actual vs Predicted

test_start_idx = WINDOW + split_idx
test_dates = ts.index[test_start_idx : test_start_idx + len(y_true)]

plt.figure(figsize=(12, 4))
plt.plot(test_dates, y_true, label="Actual (Test)", linewidth=2)
plt.plot(test_dates, y_pred, label="Predicted (CNN)", linestyle="--")
plt.title("Q2.2 — Actual vs Predicted Close (Test Set, Original Scale)")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Residuals over time
errors = y_true - y_pred
plt.figure(figsize=(12, 3))
plt.plot(test_dates, errors)
plt.title("Q2.2 — Prediction Errors (Actual − Predicted)")
plt.xlabel("Date")
plt.ylabel("Error")
plt.grid(True)
plt.tight_layout()
plt.show()

#**Q2.3** - Answer

The 1D-CNN shows good capability in capturing short-term dependencies, as reflected by its relatively low MAE (18) and RMSE (22) — indicating that it learns local fluctuations within the 10-day input window quite effectively.

However, it does not capture long-term trends as well, because a CNN processes only a fixed and local window of recent history and does not retain information beyond that window. This architectural limitation prevents it from fully modeling broader directional movements over time.


### Question 3 - Recurrent Neural Networks (RNN) (20%)
- Q3.1 Create an RNN model for time series forecasting. The model should include a SimpleRNN layer of size 40 and ReLU activation. Train the model using the provided dataset for 50 epochs. Use the batch_size of 16, and ADAM optimizer. Plot the training loss and validation loss. (12%)
- Q3.2 Evaluate the model's performance using RMSE and MAE on the test set. Plot the predicted values alongside the actual values to visualize how well the model is forecasting the time series. (6%)
- Q3.3 Why did we use a smaller batch size compared to CNN model. Why might RNN struggle with long-term dependencies, and how is this reflected in the evaluation metrics? (2%)


In [ ]:
from tensorflow.keras import layers, models, optimizers

# 1) Prepare shapes for RNN
#    - RNN expects (samples, timesteps, features) just like Conv1D.

WINDOW = X_train.shape[1]

X_train_rnn = X_train.reshape(-1, WINDOW, 1)
X_test_rnn  = X_test.reshape(-1, WINDOW, 1)

y_train_vec = y_train.ravel()
y_test_vec  = y_test.ravel()

print("Q3.1 — Shapes ready for RNN")
print(f"X_train_rnn: {X_train_rnn.shape}  (samples, timesteps={WINDOW}, features=1)")
print(f"X_test_rnn:  {X_test_rnn.shape}")
print(f"y_train:     {y_train_vec.shape}")
print(f"y_test:      {y_test_vec.shape}")

# 2) Chronological validation split from the training set (last 10% as validation)

val_start = int(len(X_train_rnn) * 0.9)
X_tr, X_val = X_train_rnn[:val_start], X_train_rnn[val_start:]
y_tr, y_val = y_train_vec[:val_start],  y_train_vec[val_start:]

print("\nQ3.1 — Train/Val split (from training set)")
print(f"X_tr:  {X_tr.shape}, y_tr:  {y_tr.shape}")
print(f"X_val: {X_val.shape}, y_val:{y_val.shape}")

# 3) Define SimpleRNN model per spec
#    - SimpleRNN(40, activation='relu')
#    - Dense(1) for regression output
tf.random.set_seed(42)
np.random.seed(42)

rnn_model = models.Sequential([
    layers.SimpleRNN(40, activation='relu', input_shape=(WINDOW, 1)),
    layers.Dense(1)
])

rnn_model.compile(optimizer=optimizers.Adam(), loss='mse', metrics=['mae'])
rnn_model.summary()

# 4) Train for 50 epochs, batch_size=16
EPOCHS = 50
BATCH_SIZE = 16

rnn_hist = rnn_model.fit(
    X_tr, y_tr,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1
)

# 5) Plot training vs validation loss (MSE)
plt.figure(figsize=(10, 4))
plt.plot(rnn_hist.history['loss'], label='Train Loss (MSE)')
plt.plot(rnn_hist.history['val_loss'], label='Validation Loss (MSE)')
plt.title("Q3.1 — Training vs. Validation Loss (SimpleRNN)")
plt.xlabel("Epoch")
plt.ylabel("Loss (MSE)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 1) Predict on the test set (still in [0,1] scale)

y_pred_scaled = rnn_model.predict(X_test_rnn, verbose=0).ravel()

# 2) Inverse-transform to original price scale

y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()
y_true = y_scaler.inverse_transform(y_test_vec.reshape(-1, 1)).ravel()

# 3) Metrics in original units

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae  = mean_absolute_error(y_true, y_pred)

print("Q3.2 — Test Metrics (original price scale)")
print(f"RMSE: {rmse:.4f}")
print(f"MAE : {mae:.4f}")

# 4) Plot Actual vs Predicted

test_start_idx = WINDOW + split_idx
test_dates = ts.index[test_start_idx : test_start_idx + len(y_true)]

plt.figure(figsize=(12, 4))
plt.plot(test_dates, y_true, label="Actual (Test)", linewidth=2)
plt.plot(test_dates, y_pred, label="Predicted (SimpleRNN)", linestyle="--")
plt.title("Q3.2 — Actual vs Predicted Close (Test Set, Original Scale)")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# Residuals over time
errors = y_true - y_pred
plt.figure(figsize=(12, 3))
plt.plot(test_dates, errors)
plt.title("Q3.2 — Prediction Errors (Actual − Predicted)")
plt.xlabel("Date")
plt.ylabel("Error")
plt.grid(True)
plt.tight_layout()
plt.show()

#**Q3.3** - Answer

**1. Why smaller batch size (16 vs 32)?**

RNNs process data sequentially and are more sensitive to temporal patterns than CNNs. A smaller batch size provides more frequent weight updates, helping the model better track subtle short-term sequence changes and avoid averaging away important temporal signals.

**2. Why RNN struggles with long-term dependencies?**

SimpleRNN has no memory gate and suffers from vanishing gradients, meaning it quickly forgets information from earlier timesteps. It mainly captures short-term behavior only. This can be seen in our metrics with the RMSE (22) and MAE (18), the model does reasonably well on short term predictions - but misses some larger or longer-term movements, confirming its limited ability to capture long-term trends.



### Question 4 - Long Short-Term Memory (LSTM) (20%)
- Q4.1 Create an LSTM model for time series forecasting.The model should include two LSTM layers of size 100 and 50, and ReLU activation. Train the model using the provided dataset for 100 epochs. Use the batch_size of 16, and ADAM optimizer. Plot the training loss and validation loss. (12%)
- Q4.2 Evaluate the model's performance using RMSE and MAE on the test set. Plot the predicted values alongside the actual values to visualize how well the model is forecasting the time series. (6%)
- Q4.3 Why do we use a larger epoch compared to RNN? (2%)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Reshape inputs for LSTM
X_train_lstm = X_train.reshape(-1, WINDOW, 1)
X_test_lstm  = X_test.reshape(-1, WINDOW, 1)
y_train_vec  = y_train.ravel()
y_test_vec   = y_test.ravel()

# Chronological validation split (last 10% of training)
val_start = int(len(X_train_lstm) * 0.9)
X_tr, X_val = X_train_lstm[:val_start], X_train_lstm[val_start:]
y_tr, y_val = y_train_vec[:val_start],  y_train_vec[val_start:]

# Build LSTM model
lstm_model = Sequential([
    LSTM(100, activation='relu', return_sequences=True, input_shape=(WINDOW,1)),
    LSTM(50, activation='relu'),
    Dense(1)
])

lstm_model.compile(optimizer=Adam(), loss='mse')

# Train model
history_lstm = lstm_model.fit(
    X_tr, y_tr,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=16,
    verbose=1
)

# Plot training vs validation loss
plt.figure(figsize=(8,4))
plt.plot(history_lstm.history['loss'], label='Train Loss')
plt.plot(history_lstm.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('LSTM Training vs Validation Loss')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Predict on the test set
y_pred_scaled = lstm_model.predict(X_test_lstm, verbose=0).ravel()
y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1,1)).ravel()
y_true = y_scaler.inverse_transform(y_test_vec.reshape(-1,1)).ravel()

# Compute metrics
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae  = mean_absolute_error(y_true, y_pred)
print(f"Test RMSE: {rmse:.4f}")
print(f"Test MAE : {mae:.4f}")

# Plot Actual vs Predicted
test_start_idx = WINDOW + split_idx
test_dates = ts.index[test_start_idx : test_start_idx + len(y_true)]

plt.figure(figsize=(10,4))
plt.plot(test_dates, y_true, label='Actual', linewidth=2)
plt.plot(test_dates, y_pred, label='Predicted', linestyle='--')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('LSTM Actual vs Predicted (Test Set)')
plt.legend()
plt.show()

# Plot residuals
errors = y_true - y_pred
plt.figure(figsize=(10,3))
plt.plot(test_dates, errors)
plt.xlabel('Date')
plt.ylabel('Error')
plt.title('LSTM Prediction Errors')
plt.show()


We use a larger number of epochs for LSTM models because they have more complex internal structures: memory cells and gates (input, forget, and output gates), which take longer to learn long-term dependencies in sequential data.
LSTMs also have more parameters than traditional RNNs, so they need more training iterations (epochs) to converge and capture temporal relationships accurately without underfitting.

### Question 5 - Gated Recurrent Unit (GRU) (20%)
- Q5.1 Create a GRU model for time series forecasting.The model should include two GRU layers of size 80 and 40, and ReLU activation. Train the model using the provided dataset for 100 epochs. Use the batch_size of 16, and ADAM optimizer. Plot the training loss and validation loss. (12%)
- Q5.2 Evaluate the model's performance using RMSE and MAE on the test set. Plot the predicted values alongside the actual values to visualize how well the model is forecasting the time series.(6%)
- Q5.3 How does the GRU compare to LSTM in terms of computational efficiency and forecasting performance? (2%)

*Write your Answer to Q5.3 Here:*

### Question 6 - Discussion (5%)
- Q6.1 Compare the performance of 1D-CNN, RNN, LSTM, and GRU models based on training and test execution time, performance (RMSE and MAE), and potential overfitting/underfitting. (3%)
- Q6.2 Discuss in which cases you would prefer using 1D-CNN vs LSTM or GRU for time series data. (2%)


*Write your Answer to Q6.2 Here:*